In [3]:
# Importing necessary libraries

# Nearest neighbors will be the v1 model for sound drip
# will give logistic regressiona nd 
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from joblib import dump
from joblib import load
import pickle
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize

# textblob will be used to perform sentiment analysis down the road
# from textblob import TextBlob

# # Data is large and will need to be unzipped
# import zipfile

# # unzipping file
# with zipfile.ZipFile("./data_collection/final_data/DF_v1.pkl.zip","r") as zip_ref:
#     zip_ref.extractall(".")

# reading in pickled data
# song_list = pd.read_pickle("./DF_v1.pkl")

In [246]:
# ALERT - THIS COULD POTENTIALLY BE USED TO PERFORM SENTIMENT ANALYSIS ON LYRICS DATA IN v2. FOR v1, DISREGARD

# # creating polarity and subjectivity lists to be populated

# lyrics_polarity = []
# lyrics_subjectivity = []

# # using a for loop to populate polarity and subjectivity lists
# for title in song_list_df["lyrics"]:
#     try:
#         blob_test = TextBlob = TextBlob(title)
#         blob_sentiment = blob_test.sentiment
#         # appending results to lists above
#         lyrics_polarity.append(blob_sentiment[0])
#         lyrics_subjectivity.append(blob_sentiment[1])
#     except:
#         lyrics_polarity.append(0)
#         lyrics_subjectivity.append(0)

# # adding polarity and subjectivity to df
# song_list['lyrics_polarity'] = lyrics_polarity
# song_list['lyrics_subjectivity'] = lyrics_subjectivity

# Loading Latest PKL

In [42]:
X = pickle.load(open("./data/song_list_v5_hashed.pkl", "rb"))

In [ ]:
# X["genres"].head()

In [ ]:
# X.columns[0:20]

# Remove two NaN Values

In [ ]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# client_credentials_manager = SpotifyClientCredentials(client_id='63594c9b2f99411a8cbd18df04851fc4',client_secret='096168b2bd1f4378ae410726955c9ed8')
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# def get_popularity(trackID):
#     return sp.track(trackID)['popularity']

In [ ]:
# get_popularity('5gNNUKhBzakQrbu8UaLori')

In [ ]:
# X.loc[202786, 'popularity'] = 18

# X.loc[202786]['popularity']

In [ ]:
# get_popularity('25DO8ImBd90EhKEKuntxqN')

In [ ]:
# X.loc[189355, 'popularity'] = 0

# X.loc[189355]['popularity']

In [ ]:
# X["genres_stripped"]

# Picking Final Updated DF

In [ ]:
# X = X.drop(["availability"], axis=1)

In [ ]:
# filename2 = './data/song_list_v5_hashed.pkl'
# pickle.dump(X, open(filename2, 'wb'))

# Preparing for Scaling

In [43]:
# dropping unnecessary columns
X = X.drop(["songid","artist","track", "duration_ms","genres","genres_stripped"], axis=1)

# checking that they're dropped
X.shape

(574018, 63)

In [44]:
columns_for_scaling = X.columns[0:13]

In [45]:
# columns_genre_hashed = X.columns[13:]

In [46]:
X[columns_for_scaling]

,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,0.001300,0.487,0.678,0.055100,9.0,0.0846,-7.780,1.0,0.0495,149.940,4.0,0.8700
1,15,0.000045,0.662,0.823,0.952000,4.0,0.3430,-1.711,0.0,0.0662,177.745,4.0,0.6210
2,17,0.002760,0.859,0.741,0.000000,11.0,0.3250,-12.329,0.0,0.2710,98.082,4.0,0.5290
3,21,0.348000,0.361,0.483,0.000003,7.0,0.1770,-6.875,1.0,0.0287,94.538,4.0,0.6820
4,1,0.340000,0.533,0.302,0.179000,10.0,0.1110,-10.308,1.0,0.0307,134.959,3.0,0.2940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
574013,33,0.798000,0.320,0.104,0.000750,0.0,0.1060,-13.162,1.0,0.0342,124.677,3.0,0.0566
574014,50,0.820000,0.418,0.204,0.936000,8.0,0.0866,-13.522,1.0,0.0370,110.022,4.0,0.0375
574015,0,0.070400,0.614,0.952,0.000000,6.0,0.3620,-2.782,1.0,0.0718,148.079,4.0,0.8200
574016,18,0.064000,0.343,0.487,0.000000,2.0,0.1070,-9.462,1.0,0.0473,171.217,4.0,0.3650


In [32]:
X_new = X[columns_for_scaling].reindex(sorted(X[columns_for_scaling].columns), axis=1)

In [33]:
X_new

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,valence
0,0.001300,0.487,0.678,0.055100,9.0,0.0846,-7.780,1.0,0,0.0495,149.940,4.0,0.8700
1,0.000045,0.662,0.823,0.952000,4.0,0.3430,-1.711,0.0,15,0.0662,177.745,4.0,0.6210
2,0.002760,0.859,0.741,0.000000,11.0,0.3250,-12.329,0.0,17,0.2710,98.082,4.0,0.5290
3,0.348000,0.361,0.483,0.000003,7.0,0.1770,-6.875,1.0,21,0.0287,94.538,4.0,0.6820
4,0.340000,0.533,0.302,0.179000,10.0,0.1110,-10.308,1.0,1,0.0307,134.959,3.0,0.2940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
574013,0.798000,0.320,0.104,0.000750,0.0,0.1060,-13.162,1.0,33,0.0342,124.677,3.0,0.0566
574014,0.820000,0.418,0.204,0.936000,8.0,0.0866,-13.522,1.0,50,0.0370,110.022,4.0,0.0375
574015,0.070400,0.614,0.952,0.000000,6.0,0.3620,-2.782,1.0,0,0.0718,148.079,4.0,0.8200
574016,0.064000,0.343,0.487,0.000000,2.0,0.1070,-9.462,1.0,18,0.0473,171.217,4.0,0.3650


In [47]:
# instantiating scaler
scaler = MinMaxScaler()

# scaling data
# data_fitted = scaler.fit(X_new)
data_scaled = scaler.fit_transform(X[columns_for_scaling])

# confirming scaling took place
data_scaled[0]

array([0.        , 0.00130522, 0.4884654 , 0.678     , 0.0551    ,
       0.81818182, 0.0846    , 0.78720453, 1.        , 0.05118925,
       0.60009846, 0.8       , 0.87      ])

In [37]:
data_fitted

MinMaxScaler(copy=True, feature_range=(0, 1))

# Joblibbing Scaler to be used in new flask app

In [38]:
dump(data_fitted,"./data/scalar3.joblib",compress=True)

['./data/scalar3.joblib']

In [ ]:
# pd.DataFrame(data_scaled,columns=X[columns_for_scaling].columns).hist(figsize=(8,8), normed=True);

In [ ]:
# data_for_model = np.concatenate((data_scaled,genres_hashed_array),axis=1)

In [73]:
# len(data_scaled)

574018

# Normalization - Necessary Part of the Production App

In [74]:
# import sys

In [75]:
# np.set_printoptions(suppress=True)

In [76]:
# data_scaled[7]

array([0.18556701, 0.85040161, 0.58375125, 0.0846    , 0.        ,
       1.        , 0.696     , 0.69854981, 1.        , 0.03660807,
       0.39780036, 0.8       , 0.396     ])

In [10]:
columns_for_normalization = columns_for_scaling

In [11]:
columns_for_normalization

Index(['popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [48]:
normalizer = Normalizer()
data_normalized = normalizer.fit_transform(data_scaled)

In [ ]:
# Valence isn't scaled appropriately
# will have to scale before normalizing
# pd.DataFrame(da,columns=columns_for_normalization).hist(figsize=(8,8), normed=True);

# K-Nearest Neighbors Classifier w/ Data scaled and Normalized

In [59]:
df = pickle.load(open("./data/song_list_v5_hashed.pkl", "rb"))

In [49]:
# instantiating model class
knn1 = NearestNeighbors(n_neighbors=5000,algorithm='kd_tree',n_jobs=-1)

# fitting model
knn1 = knn1.fit(data_normalized)

# Dump Model If n_neighbors param needs to be adjusted

In [254]:
# dump(model_1, 'model3.joblib', compress=True)

# Song_ID Array for Predict Flask Function

In [255]:
# # converting song id column from df to an array

# song_id_array3 = np.array(df['songid'])

# filename5 = './data/song_id_array3.pkl'
# pickle.dump(song_id_array3, open(filename5, 'wb'))

# Loading Joblibbed Model

In [40]:
knn1 = load('./data/model3.joblib')

In [50]:
# running model to test output
knn_results1 = knn1.kneighbors([data_normalized[2]])[1]

In [51]:
data_normalized[2].shape

(13,)

In [52]:
knn_results1[0][0:20]

array([     2,  39565, 265926, 532028, 407709, 438331, 473470, 259888,
       420313, 213790, 352801, 303257, 551492, 106525, 452963,  64608,
       124604,  46919, 433612, 572160])

In [53]:
len(knn_results1[0][0:])

5000

In [56]:
pd.DataFrame(X.loc[[3]][0:18])

,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,...,40,41,42,43,44,45,46,47,48,49
3,21,0.348,0.361,0.483,0.000003,7.0,0.177,-6.875,1.0,0.0287,...,0.0,-2.0,1.0,2.0,-1.0,2.0,0.0,0.0,0.0,0.0


In [18]:
pd.set_option('display.max_colwidth', -1)

/Users/user/.local/share/virtualenvs/model_notebooks-inW34qmC/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [60]:
counter = 0
for result in knn_results1[0][0:20]:
    counter += 1
    print(counter)
    display(pd.DataFrame(df.loc[result][0:1]),pd.DataFrame(df.loc[result][1:2]),pd.DataFrame(df.loc[result][17:18]))

1


,2
songid,3GsS8jzoixpCnp4jDWCEvb


,2
popularity,17


,2
genres,"['hip hop', 'new jack swing', 'rap']"


2


...

,433612
genres,"['dub', 'lovers rock', 'old school dancehall',..."


20


,572160
songid,3Whyr0c7Iji5UhZOn8OMBz


,572160
popularity,13


,572160
genres,"['dance pop', 'pop', 'post-teen pop', 'viral p..."


# Model w/ Scaling and no Normalization

In [ ]:
# instantiating model class
# knn2 = NearestNeighbors(n_neighbors=100,leaf_size=10,algorithm='kd_tree',n_jobs=-1)

# fitting model
# model2 = knn2.fit(data_scaled)

In [ ]:
# running model to test output
# knn_results2 = knn2.kneighbors([data_scaled[1]])[1]

In [ ]:
# knn_results2[0]

In [ ]:
# pd.DataFrame(df.loc[1][1:18])

In [ ]:
# counter = 0 
# for result in knn_results2[0]:
#     counter += 1 
#     print(counter)
#     display(pd.DataFrame(df.loc[result][0:1]),pd.DataFrame(df.loc[result][17:18]))

# OUTDATED -  Using Dataframe - Filtering KK For Shared Genres

In [239]:
#loop takes KNN results and filters by source track genres
# old
filtered_list = []
source_song_index = knn_results1[0][0]
source_genre_list = df.loc[source_song_index]["genres_stripped"].split(",")
# print(source_genre_list)
for source_genre in source_genre_list:
    source_genre = source_genre.strip(' ')
#     print("source_genre_created")
#     print(source_genre)
    for output_song_index in knn_results1[0][1:2]:
        print(output_song_index)
#         output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#         print("output_genre_list created")
#         print(output_genre_list)
#         for output_genre in output_genre_list:
#             output_genre = output_genre.strip(' ')
#             print("output_genre_created")
#             print(output_genre)
#             if source_genre == output_genre:
#                 print(output_song_index,source_genre,output_genre)
#                 filtered_list.append(output_song_index)
#                 print(filtered_list)
# #         filtered_lists.append(filtered_list)
# #         print(filtered_lists,"!!!!!")
    

103114
103114
103114


# Attemping another approach using dataframe - success

In [306]:
#loop takes KNN results and filters by source track genres
# old
filtered_list = []
# filtered_lists = []
source_song_index = knn_results1[0][0]
source_genre_list = df.loc[source_song_index]["genres_stripped"].split(",")
# print(source_genre_list)
for output_song_index in knn_results1[0][1:]:
#     print(output_song_index)
    output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#     print(output_genre_list)
    for output_genre in output_genre_list:
        output_genre = output_genre.strip(' ')
#         print(output_genre)
        for source_genre in source_genre_list:
            source_genre = source_genre.strip(' ')
#             print(source_genre)
            if source_genre == output_genre:
#                 print(output_song_index,source_genre,output_genre)
                filtered_list.append(output_song_index)
#             print(filtered_list)
            else:
# #             print("source genre does not match output genre!")
                continue
#     filtered_lists.append(filtered_list)
print(len(set(filtered_list)),"!!!!!")

807 !!!!!


In [260]:
len(set(filtered_list))

376

In [308]:
knn_results1[0][0]

2

# Pickle Processed Genres Array for Use in Genre Filtering Function

In [174]:
genre_array = load("./data/genres_array.pkl")

In [190]:
genres_list_2 = []

for row in range(0,574018):
    g_list = [genre.strip(" ") for genre in genre_array[row]]
    genres_list_2.append(g_list)

In [194]:
genre_array = np.array(genres_list_2)

In [198]:
# pickle.dump(genre_array,open("./data/genres_array_2.pkl","wb"))

# Load in Processed Genres Array

In [63]:
genre_array = load("./data/genres_array_2.pkl")

# Ready for Production - Load in Genre Array and Utilize in updated function for genre filtering

In [66]:
def filter_model(): 
    #loop takes KNN results and filters by source track genres
    filtered_list = []
    song_list_length = 364
    source_song_index = knn_results1[0][0]
    source_genre_list = genre_array[source_song_index]
#     for output_song_index in knn_results1[0][1:]:
#         output_genre_list = genre_array[output_song_index]
#         for output_genre in output_genre_list:
#             for source_genre in source_genre_list:
#                 if source_genre == output_genre:
#                     filtered_list.append(output_song_index)
#                 else:
#                     continue
#     if len(set(filtered_list)) > song_list_length:
#         print("list large enough")
#     else:
#         counter = song_list_length - len(set(filtered_list))
#         print(len(set(filtered_list)))
#         print(counter)
#         print(f'need to add {counter} items to final song output')
#         for output_song_index in knn_results1[0][1:]:
#             if output_song_index not in filtered_list:
#                 if counter > 0:
#                     filtered_list.append(output_song_index)
#                     counter -= 1
#                 else:
#                     break
            
# #     print(len(output_song_index_list),"!!!!!")    
#     print(len(set(filtered_list)),"!!!!!")

In [67]:
filter_model()

["'hip hop'", "'new jack swing'", "'rap'"]


In [ ]:
filtered_list

In [281]:
len(knn_results1[0][1:])

9999

# Test Multiple Rows - Functional!

In [272]:
def output_len_of_lists(list_of_lists):
    for list_item in list_of_lists:
        print(len(list_item))

In [ ]:
#loop takes range of KNN results per defined track range and filter matches by source track genres
filtered_lists = []
for source_song_index in range(0,1000):
    knn_results1 = knn1.kneighbors([data_normalized[source_song_index]])[1]
    knn_result_ouputs = knn_results1[0][1:]
    filtered_list = []
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_result_ouputs:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    filtered_lists.append(set(filtered_list))
    
output_len_of_lists(filtered_lists)

# Analysis on Amount of Tracks that will have at least 20 genre matching nearest neighbors

In [320]:
nineteen_tracks = 0 
for filtered_list_temp in filtered_lists:
    if len(filtered_list_temp) > 19:
        nineteen_tracks += 1
print(nineteen_tracks / 1000)

0.814


# Pickling model - Outdated

In [ ]:
import pickle

filename = './models/knn_model_v1.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# pulling in the (now) pickled model
model_pickle = pickle.load(open(filename, 'rb'))

In [ ]:
# Loading model to test on scaled data (not pickled data)
pickle_results = model_pickle.kneighbors([data_scaled[1000]])[1]

# displaying output
pickle_results

# Pickling Scaled Data - Outdated

In [35]:
# filename2 = './data_scaled.pkl'
# pickle.dump(data_scaled, open(filename2, 'wb'))

# (OLD) Testing both pickles for dataset, model

In [ ]:
# importing libraries (even though they are visible above)
# to illustrate which packages are needed to create the function below

import pandas as pd 
from sklearn.neighbors import NearestNeighbors
import pickle


# this function predicts on a song id

def predict(id):
    #loads in pickled data and model
    X = pickle.load(open('./data_scaled.pkl', 'rb'))
    loaded_pickle = pickle.load(open('./models/knn_model_v1.pkl', 'rb'))
    #calculates results
    results = loaded_pickle.kneighbors([X[id]])[1]
    return results

In [ ]:
predict(0)

In [ ]:
content = {
    "audio_features": {
        "acousticness": 0.934,
        "danceability": 0.186,
        "energy": 0.107,
        "instrumentalness": 0,
        "key": 5,
        "liveness": 0.297,
        "loudness": -14.802,
        "mode": 1,
        "speechiness": 0.0347,
        "tempo": 107.095,
        "time_signature": 4,
        "valence": 0.149
    }
}

In [ ]:
dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')

In [ ]:
dataframe

In [ ]:
dataframe_scaled = scaler.transform(dataframe)

dataframe_scaled

In [ ]:
import pickle

filename = './models/scaler.pkl'
pickle.dump(scaler, open(filename, 'wb'))

In [ ]:
target = 'songid'

song_id_array = song_list[target]

In [ ]:
song_id_array

In [ ]:
sys.getsizeof(song_id_array)

In [ ]:
song_id_list = song_id_array.tolist()

In [ ]:
sys.getsizeof(song_id_list)

In [ ]:
import pickle

filename = 'Flask_API/SOUNDDRIP/data/song_id_array.pkl'
pickle.dump(song_id_array, open(filename, 'wb'))

In [ ]:
import pickle

filename = 'song_id_list2.pkl'
pickle.dump(song_id_list, open(filename, 'wb'))

In [ ]:
content = {
    "audio_features": {
        "acousticness": 0.934,
        "danceability": 0.186,
        "energy": 0.107,
        "instrumentalness": 0,
        "key": 5,
        "liveness": 0.297,
        "loudness": -14.802,
        "mode": 1,
        "speechiness": 0.0347,
        "tempo": 107.095,
        "time_signature": 4,
        "valence": 0.149
    }
}

def predict(content):
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scaler = pickle.load(open('models/scaler.pkl', 'rb'))
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scaler.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = pickle.load(open('./models/knn_model_v1.pkl', 'rb'))
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')

    return results


In [ ]:
predict(content)

In [ ]:
  # For-loop for returning 20 songs
def all_similarities(data_result):
    similar_songs = []
    print('song_id_list loading...')
    song_id_list = pickle.load(open('Flask_API/SOUNDDRIP/data/song_id_list.pkl', 'rb'))
    print('song_id_list loaded')
    print('beginning for loop...')
    
    for song_row in data_result[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict

In [ ]:
all_similarities(predict(content))

In [ ]:
sorted_list = sorted(similarities, key=lambda i: i['similarity'], 
                                                         reverse=True)[:20]
      json_dict = {"songs": sorted_list}
      #data = json.dumps(json_dict)
      return jsonify(json_dict), print('yay')

# Get Spotify API Token / Output Audio Features

In [ ]:
import spotipy
import spotipy.util as util

In [ ]:
USERNAME = 'augt12ocf9csxa4s8kbq9reg8' #your spotify username
CLIENT_ID = 'b954b92b9a674e6a9cf7322649da82d1' #set at your developer account
CLIENT_SECRET = '920cd41023994b1a99210243ce8967b2' #set at your developer account
REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

In [ ]:
token

In [ ]:
token_content = {'token': token}

In [ ]:
type(token_content)

In [ ]:
token_content['token']

In [ ]:
def get_id(token):
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    song_id = results['items'][0]['track']['id']
    return song_id

In [ ]:
def get_features(song_id):
    results_dict = sp.audio_features(song_id)[0]
    audio_features = {"audio_features":{key: results_dict[key] for key in results_dict.keys() & {'danceability', 'energy',
                                                                                                'key', 'loudness', 'mode',
                                                                                                'speechiness', 'acousticness',
                                                                                                'instrumentalness', 'liveness',
                                                                                                'valence', 'tempo', 'time_signature'}}}
        
    return audio_features

In [ ]:
 acoustical_features = get_features(get_id(token))

In [ ]:
acoustical_features

# Flask Predict function

In [ ]:
def predict(content):
    similar_songs = []
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scalar = load('scalar2.joblib')
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scalar.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = load('model2.joblib')
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')
    print('song_id_list loading...')
    song_id_list = load('song_id_list2.joblib')
    print('song_id_list loaded')
    
    print('beginning for loop...')
    for song_row in results[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from flask import request
from joblib import dump
from joblib import load

import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify

In [ ]:
%timeit

predict(acoustical_features)

# Dump

In [ ]:
dump(model, 'model2.joblib', compress=True)

In [ ]:
# %time

# pickle_loaded = pickle.load(open('Flask_API/SOUNDDRIP/models/knn_model_v1.pkl', 'rb'))
# joblib_loaded = load('model.joblib')



In [ ]:
# dump(data_scaled, 'data_scaled.joblib', compress=True)

In [ ]:
dump(scaler, 'scalar2.joblib', compress=True)

In [ ]:
dump(song_id_list, 'song_id_list2.joblib', compress=True)